In [ ]:
!pip install git+https://github.com/aleju/imgaug

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as T
import torchvision.datasets as D
from imgaug import augmenters as iaa
import imgaug as ia
import numpy as np

class MiRuido:
  def __init__(self):
    self.aug = iaa.SaltAndPepper(0.1)
    
  def __call__(self, img):
    img = np.array(img)
    return self.aug.augment_image(img)
  

dataset_train = D.FashionMNIST('./dataset', download=True, train=True, 
                               transform=T.ToTensor())
dataset_test = D.FashionMNIST('./dataset', download=True, train=False,
                              transform = T.ToTensor())

dataset_train_x = D.FashionMNIST('./dataset', download=False, train=True,
                                transform=T.Compose([MiRuido(),T.ToTensor()]))
dataset_test_x = D.FashionMNIST('./dataset', download=False, train=False,
                               transform=T.Compose([MiRuido(), T.ToTensor()]))

train_loader = torch.utils.data.DataLoader(dataset_train, 
                                           batch_size=16, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset_test, 
                                           batch_size=16, shuffle=False)

train_loader_x = torch.utils.data.DataLoader(dataset_train_x, 
                                           batch_size=16, shuffle=False)
test_loader_x = torch.utils.data.DataLoader(dataset_test_x, 
                                           batch_size=16, shuffle=False)



In [ ]:
#Inspeccion
it1 = iter(train_loader)
data1, target1 = next(it1)

it2 = iter(train_loader_x)
data2, target2 = next(it2)

import matplotlib.pyplot as plt

grid1 = torchvision.utils.make_grid(data1)
plt.imshow(grid1.permute(1, 2, 0))
plt.show()

grid2 = torchvision.utils.make_grid(data2)
plt.imshow(grid2.permute(1, 2, 0))
plt.show()




In [ ]:
class NoiseAutoencoder(nn.Module):
  def __init__(self):
    super(NoiseAutoencoder, self).__init__()
    
    self.encoder = nn.Sequential(nn.Conv2d(1, 16, kernel_size=3, padding=1),
                                nn.ReLU(),
                                nn.MaxPool2d(2,2),
                                nn.Conv2d(16, 32, kernel_size = 3, padding=1),
                                nn.ReLU(),
                                nn.MaxPool2d(2, 2),
                                nn.Conv2d(32, 16, kernel_size=3, padding=1),
                                nn.ReLU(),
                                nn.MaxPool2d(2, 2, padding=1))
    self.decoder = nn.Sequential(nn.Conv2d(16, 16, kernel_size=3, padding=1),
                                nn.ReLU(),
                                nn.Upsample(scale_factor=2),
                                nn.Conv2d(16,32, kernel_size=3, padding=1),
                                nn.ReLU(),
                                nn.Upsample(scale_factor=2),
                                nn.Conv2d(32, 64, kernel_size=3),
                                nn.ReLU(),
                                nn.Upsample(scale_factor=2),
                                nn.Conv2d(64, 1, kernel_size=3, padding=1))
    
  def forward(self, x):
    return self.decoder(self.encoder(x))
  

net = NoiseAutoencoder()
net = net.to('cuda')
  
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr = 0.001)

In [ ]:
for epoch in range(10):
  for x,y in zip(train_loader, train_loader_x):
    x = x[0]
    y = y[0]
    
    x = x.to('cuda')
    y = y.to('cuda')
    
    output = net(y)
    loss = criterion(output, x)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
  print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 10, loss.item()))

In [ ]:
data1,_ = next(iter(test_loader))
data2,_ = next(iter(test_loader_x))

with torch.no_grad():
  out = net(data2.to('cuda'))
  
grid1 = torchvision.utils.make_grid(data1)
plt.imshow(grid1.permute(1, 2, 0))
plt.show()

grid2 = torchvision.utils.make_grid(data2)
plt.imshow(grid2.permute(1, 2, 0))
plt.show()

grid3 = torchvision.utils.make_grid(out.cpu())
plt.imshow(grid3.permute(1, 2, 0))
plt.show()
  


In [ ]:
!wget http://www.ivan-sipiran.com/downloads/noisy_docs_data.zip

In [ ]:
!unzip noisy_docs_data.zip

In [ ]:
import cv2

img = cv2.imread('noisy_docs_data/train_cleaned/101.png')
plt.imshow(img)
plt.show()

In [ ]:
import PIL
import glob

class MiDataset(torch.utils.data.Dataset):
  def __init__(self, root, tform=None, imgloader=PIL.Image.open):
    super(MiDataset, self).__init__()
    
    self.root = root
    self.filenames = sorted(glob.glob(root + '/*.png'))
    self.tform = tform
    self.imgloader = imgloader
    
  def __len__(self):
    return len(self.filenames)
  
  def __getitem__(self, i):
    out = self.imgloader(self.filenames[i])
    if self.tform:
      out = self.tform(out)
    return out

In [ ]:
doc_train_dataset = MiDataset(root='noisy_docs_data/train',
                             tform=T.Compose([T.Resize((258, 540)), 
                                              T.ToTensor()]))

doc_train_loader = torch.utils.data.DataLoader(doc_train_dataset,
                                               batch_size=4)

doc_target_dataset = MiDataset(root='noisy_docs_data/train_cleaned',
                             tform=T.Compose([T.Resize((258, 540)), 
                                              T.ToTensor()]))

doc_target_loader = torch.utils.data.DataLoader(doc_target_dataset,
                                               batch_size=4)

In [ ]:
class BackgroundRemoval(nn.Module):
  def __init__(self):
    super(BackgroundRemoval, self).__init__()
    
    self.encoder = nn.Sequential(
        nn.Conv2d(1, 64, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2))
    
    self.decoder = nn.Sequential(
        nn.Conv2d(64, 64, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.Upsample(scale_factor=2),
        nn.Conv2d(64, 1, kernel_size=3, padding=1))
    
  def forward(self,x):
    return self.decoder(self.encoder(x))
  
net2 = BackgroundRemoval()
net2 = net2.to('cuda')
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(net2.parameters(), lr=0.001)


In [ ]:
for epoch in range(100):
  for x,y in zip(doc_train_loader, doc_target_loader):
    x = x.to('cuda')
    y = y.to('cuda')
    
    output = net2(x)
    loss = criterion(output, y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
  print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 10, loss.item()))

In [ ]:
doc_test_dataset = MiDataset(root='noisy_docs_data/test',
                             tform=T.Compose([T.Resize((258, 540)), 
                                              T.ToTensor()]))

doc_test_loader = torch.utils.data.DataLoader(doc_test_dataset,
                                               batch_size=4)

data3 = next(iter(doc_test_loader))
grid3 = torchvision.utils.make_grid(data3, nrow=1)
plt.figure(figsize=(12, 12))
plt.imshow(grid3.permute(1, 2, 0))
plt.show()

In [ ]:
with torch.no_grad():
  out = net2(data3.to('cuda'))
  
grid4 = torchvision.utils.make_grid(out.cpu(), nrow=1)
plt.figure(figsize=(12, 12))
plt.imshow(grid4.permute(1, 2, 0))
plt.show()